<a href="https://colab.research.google.com/github/spmaurya143ss/LearningSpark/blob/main/Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install pyspark

In [17]:
from pyspark.sql.session import SparkSession

In [18]:
spark =  SparkSession.builder.appName("DisplayDataFrame").getOrCreate()

In [19]:
data = [('Shivam',25,'M'),('Hritik',24,'M'),('Shweta',24,'F')]
df = spark.createDataFrame(data, ['Name', 'Age','Gender'])

In [20]:
df.show()

+------+---+------+
|  Name|Age|Gender|
+------+---+------+
|Shivam| 25|     M|
|Hritik| 24|     M|
|Shweta| 24|     F|
+------+---+------+



In [21]:
myRange = spark.range(1000).toDF('Number')
# myRange.show()

In [22]:
divBy2 = myRange.where("Number % 2 = 0")
# divBy2.show()

In [23]:
divBy2.count()

500

In [24]:
!git clone https://github.com/databricks/Spark-The-Definitive-Guide

Cloning into 'Spark-The-Definitive-Guide'...
remote: Enumerating objects: 2035, done.
remote: Total 2035 (delta 0), reused 0 (delta 0), pack-reused 2035 (from 1)
Receiving objects: 100% (2035/2035), 523.88 MiB | 20.47 MiB/s, done.
Resolving deltas: 100% (305/305), done.
Updating files: 100% (1738/1738), done.


In [25]:
%cd Spark-The-Definitive-Guide/data/flight-data/csv/

/content/Spark-The-Definitive-Guide/data/flight-data/csv


In [26]:
flightData2015 = spark.read.option("inferSchema","true").option("header","true").csv("Spark-The-Definitive-Guide/data/flight-data/csv")
# flightData2015.show()

In [27]:
flightData2015.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=264),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=69)]

In [28]:
flightData2015.sort("count").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#5790 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(count#5790 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=241]
      +- FileScan csv [DEST_COUNTRY_NAME#5788,ORIGIN_COUNTRY_NAME#5789,count#5790] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/content/Spark-The-Definitive-Guide/data/flight-data/csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [29]:
spark.conf.set("spark.sql.shuffle.partitions", "200")

In [30]:
flightData2015.take(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=264)]

In [31]:
flightData2015.createOrReplaceTempView("Flight_Data_2015")

In [32]:
sqlWay = spark.sql("select DEST_COUNTRY_NAME, count(*) as Count from Flight_Data_2015 group by DEST_COUNTRY_NAME")#.where("Count = 2")
# sqlWay.show()

In [33]:
sqlWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#5788], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#5788, 200), ENSURE_REQUIREMENTS, [plan_id=263]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#5788], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#5788] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/content/Spark-The-Definitive-Guide/data/flight-data/csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>




In [34]:
dataFrameWay = flightData2015.groupBy("DEST_COUNTRY_NAME").count()
# dataFrameWay.show()

In [35]:
dataFrameWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#5788], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#5788, 200), ENSURE_REQUIREMENTS, [plan_id=276]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#5788], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#5788] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/content/Spark-The-Definitive-Guide/data/flight-data/csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>




In [36]:
sqlWayMax = spark.sql("select max(count) from Flight_Data_2015")
# sqlWayMax.show()

In [37]:
from pyspark.sql.functions import max

In [38]:
flightData2015.select(max("count")).take(1)

[Row(max(count)=370002)]

In [39]:
maxSql = spark.sql("select DEST_COUNTRY_NAME, sum(count) as destination_total from Flight_Data_2015 group by DEST_COUNTRY_NAME order by sum(count) DESC limit 5")
maxSql.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|          2348280|
|           Canada|            49052|
|           Mexico|            38075|
|   United Kingdom|            10946|
|            Japan|             9205|
+-----------------+-----------------+



In [40]:
from pyspark.sql.functions import desc

In [41]:
max5= flightData2015.groupBy("DEST_COUNTRY_NAME").sum("count").withColumnRenamed("sum(count)", "destination_total").sort(desc("destination_total")).limit(5)
# max5.show()

In [42]:
max5.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- TakeOrderedAndProject(limit=5, orderBy=[destination_total#5854L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#5788,destination_total#5854L])
   +- HashAggregate(keys=[DEST_COUNTRY_NAME#5788], functions=[sum(count#5790)])
      +- Exchange hashpartitioning(DEST_COUNTRY_NAME#5788, 200), ENSURE_REQUIREMENTS, [plan_id=369]
         +- HashAggregate(keys=[DEST_COUNTRY_NAME#5788], functions=[partial_sum(count#5790)])
            +- FileScan csv [DEST_COUNTRY_NAME#5788,count#5790] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/content/Spark-The-Definitive-Guide/data/flight-data/csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>




In [43]:
staticDataFrame = spark.read.format("CSV").option("header","true").option("inferSchema",'true').load("Spark-The-Definitive-Guide/data/retail-data/by-day/*.csv")

In [44]:
staticDataFrame.createOrReplaceTempView("retail_data")

In [45]:
staticSchema = staticDataFrame.schema

In [46]:
from pyspark.sql.functions import window, column, desc, col


In [47]:
oneDayDF = staticDataFrame.selectExpr("CustomerId", "(UnitPrice * Quantity) as total_cost","InvoiceDate").groupby( col("CustomerId"), window(col("InvoiceDate"), "1 day")).sum("total_cost")
# oneDayDF.show()

In [48]:
streamingDataFrame = spark.readStream.schema(staticSchema).option("maxFilesPerTrigger", 1).format("csv").option("header", "true").load("Spark-The-Definitive-Guide/data/retail-data/by-day/*.csv")

In [49]:
streamingDataFrame.isStreaming

True

In [50]:
purchasedByCustomerPerHour = streamingDataFrame.selectExpr("CustomerId", "(UnitPrice * Quantity) as total_cost","InvoiceDate").groupby( col("CustomerId"), window(col("InvoiceDate"), "1 day")).sum("total_cost")

In [51]:
purchasedByCustomerPerHour.writeStream.format("console").queryName("customer_purchases2").outputMode("complete").start()

In [52]:
staticDataFrame.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [53]:
from pyspark.sql.functions import date_format, col
preppedDataFrame = staticDataFrame.na.fill(0).withColumn("day_of_week", date_format(col("InvoiceDate"), "EEEE")).coalesce(5)
# preppedDataFrame.show()

In [54]:
trainDataFrame = preppedDataFrame.where("InvoiceDate < '2011-07-01'")
testDataFrame = preppedDataFrame.where("InvoiceDate >= '2011-07-01'")

In [55]:
trainDataFrame.count()

245903

In [56]:
testDataFrame.count()

296006

In [57]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer().setInputCol("day_of_week").setOutputCol("day_of_week_index")
# indexer.show()

In [58]:
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder().setInputCol("day_of_week_index").setOutputCol("day_of_week_encoded")

In [59]:
from pyspark.ml.feature import VectorAssembler
vectAssembler = VectorAssembler().setInputCols(["UnitPrice", "Quantity", "day_of_week_encoded"]).setOutputCol("features")

In [60]:
from pyspark.ml import Pipeline
transformationPipeline = Pipeline().setStages([indexer, encoder,vectAssembler])

In [61]:
fittedPipeline = transformationPipeline.fit(trainDataFrame)

In [62]:
transformedTraining = fittedPipeline.transform(trainDataFrame)

In [63]:
transformedTraining.cache()

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: timestamp, UnitPrice: double, CustomerID: double, Country: string, day_of_week: string, day_of_week_index: double, day_of_week_encoded: vector, features: vector]

In [64]:
from pyspark.ml.clustering import KMeans
kmeans = KMeans().setK(20).setSeed(11)

In [ ]:
kmModel = kmeans.fit(transformedTraining)

In [ ]:
kmModel.computeCost(transformedTraining)